In [18]:
import sys
import os
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
import networkx as nx
import pysindy as ps
import scipy as sp
from scipy import sparse
from scipy.spatial.distance import pdist
from sklearn.linear_model import Lasso
from sklearn.metrics import pairwise_distances

def network_generate(n, eta):
    G = nx.scale_free_graph(n, alpha=0.2, beta=0.3, gamma=0.5)  # obtain a directed graph
    # remove self loops
    G.remove_edges_from(nx.selfloop_edges(G))
    # remove repeated links
    removed_list = []
    for edge in G.edges:
        if edge[2] != 0:
            removed_list.append(edge)
    G.remove_edges_from(removed_list)
    for edge in range(G.number_of_edges()):
        list(G.edges(data=True))[edge][2]["weight"] = np.random.uniform(1.-eta, 1.+eta)
    A = nx.adjacency_matrix(G).todense().T
    k_in = np.zeros(G.number_of_nodes())
    for node in range(G.number_of_nodes()):
        if G.in_degree(node) != 0:
            k_in[node] = sum(list(G.in_edges(node, data=True))[i][2]["weight"] for i in range(G.in_degree(node)))
    # weighted laplacian matrix
    L = np.diag(k_in) - A
    delta = np.max(k_in)
    return G, A, k_in, L, delta

def undirected_network(n):
    G = nx.barabasi_albert_graph(n, 2)
    A = nx.adjacency_matrix(G).todense()
    L = nx.laplacian_matrix(G).todense()
    degrees = np.array(G.degree)[:, 1]
    delta = np.max(degrees)
    return G, A, L, degrees, delta

def sync_error(x):
    err = map(lambda t: np.mean(pdist(x[:,:,t])), np.arange(x.shape[2]))
    return np.array(list(err))

def data_generate(n, m, transient, time, beta, mu, sigma, C, L, delta, h, gamma, x_noise):
    def rulkov_map(x):
        x = x.reshape(n, m).T
        return np.asarray([
            beta/(1+x[0]**2)+x[1],
            x[1]-mu*x[0]-sigma
        ]).T.flatten()

    def net_dynamics(x):
        return rulkov_map(x) - (C/delta)*sparse.kron(L, h).dot(x) + x_noise*gamma*np.random.uniform(-1, 1, n*m)

    x = np.zeros([2, n, time])
    x[0,:,0] = np.random.uniform(0.0, 1.0, n)
    x[1,:,0] = np.random.uniform(0.0, 1.0, n)
    x0 = np.vstack((x[0,:,0], x[1,:,0])).flatten()
    # transient for the isolated dynamics
    for l in range(transient):
        x0 = rulkov_map(x0)
    # transient for the coupled dynamics
    for l in range(transient):
        x0 = net_dynamics(x0)
    x = np.zeros([m*n, time])
    x[:,0] = x0
    for l in range(time-1):
        x[:,l+1] = net_dynamics(x[:,l])
    return x.reshape(n, m, time)

def data_split(n, x):
    dx = []
    X = []
    for i in range(n):
        dx.append(x[i,:,1:])
        X.append(x[i,:,:-1])
    return np.array(X), np.array(dx)

def predicted_models(n, X, dx):
    library_functions = [
        lambda x: np.sin(x),
        lambda x: np.cos(x),
        lambda x: 1/(1-x),
        lambda x: 1/(1-x**2),
        lambda x: 1/((1-x)**2),
        lambda x: 1/(1+x),
        lambda x: 1/(1+x**2),
        lambda x: 1/((1+x)**2),
        lambda x: 1/x,
    ]
    library_function_names = [
        lambda x: 'sin(' + x + ')',
        lambda x: 'cos(' + x + ')',
        lambda x: '1/1-' + x,
        lambda x: '1/1-' + x + '^2',
        lambda x: '1/(1-' + x + ')^2',
        lambda x: '1/1+' + x,
        lambda x: '1/1+' + x + '^2',
        lambda x: '1/(1+' + x + ')^2',
        lambda x: '1/' + x,
    ]
    
    lib_custom = ps.CustomLibrary(library_functions=library_functions, function_names=library_function_names)
    lib_poly = ps.PolynomialLibrary(degree=1, include_bias=True, include_interaction=False)
    # Simplified library combination without inputs_per_library
    lib = ps.GeneralizedLibrary([lib_custom, lib_poly])

    coeff = []
    t = np.arange(X[0].shape[1])  # Time array for discrete steps
    for i in range(n):
        model = ps.SINDy(feature_library=lib, optimizer=Lasso(alpha=0.0001, fit_intercept=False,
                                                              max_iter=100000), discrete_time=True)
        model.fit(X[i].T, t=t, x_dot=dx[i].T)
        coeff.append(model.coefficients())
    return np.array(coeff)

def similarity(n, x, coeff, k_in):
    # Check the correlation between ground-truth time series u-component
    corr_matrix_gt = np.corrcoef(x[:,0,:], x[:,0,:])[0:n,0:n]
    distance_matrix = pairwise_distances(coeff[:,0,:], metric='seuclidean')
    # similarity analysis
    s = np.sum(distance_matrix, axis=1)
    s_gt = np.sum(np.abs(corr_matrix_gt), axis=1)
    # predicted low-degree node
    print('one of the low degree nodes:', np.argmin(s))
    # predicted hub
    print('predicted hub:', np.argmax(s))
    # check
    print('Predicted low degree node', np.argmin(s), 'has', k_in[np.argmin(s)], 'in connection(s).')
    print('The real hub is:', np.argmax(k_in))
    hub_id = np.argmax(s)
    ld_id = np.argmin(s)
    return corr_matrix_gt, distance_matrix, s, s_gt, hub_id, ld_id

def local_dynamics_function(n, m, time, x, beta, mu, sigma):
    def rulkov_map(x):
        x = x.reshape(n, m).T
        return np.asarray([beta/(1+x[0]**2)+x[1], x[1]-mu*x[0]-sigma]).T.flatten()

    y = x.reshape(n*m, time)
    F_x = np.zeros_like(y)
    for i in range(time):
        F_x[:,i] = rulkov_map(y[:,i])
    Fx = F_x.reshape(n, m, time)
    return Fx

def coupling_effect(dx, hub_id, Fx):
    Y_hub = dx[hub_id,:,:] - Fx[hub_id,:,:-1]
    Y = dx[:,:,:] - Fx[:,:,:-1]
    return Y_hub, Y

def reconstruction(n, m, time, x, y):
    L_predicted = []
    model = ps.SINDy(feature_library=ps.IdentityLibrary(), optimizer=Lasso(alpha=0.001, fit_intercept=False))
    t = np.arange(time-1)  # Time array for the reconstruction step
    model.fit(x.reshape(n*m, time-1).T, t=t, x_dot=y.reshape(n*m, time-1).T)
    L_predicted.append(model.coefficients())
    L_predicted = np.array(L_predicted)[0,:,:]
    # remove kronecker product
    xxx = np.arange(0, n*m, m)
    xx, yy = np.meshgrid(xxx, xxx)
    L_predicted = L_predicted[xx, yy]
    return L_predicted

def rmse(L_true, L_pred, C, delta):
    # Compute RMSE adjusted by coupling strength C and maximum degree delta
    diff = L_true - L_pred
    mse = np.mean(diff ** 2)
    rmse_value = np.sqrt(mse) * (C / delta)  # Scaling as per original intent
    return rmse_value

def compute_performance_metrics(L_true, L_pred):
    # Threshold to determine connections (e.g., non-zero elements)
    threshold = 1e-10  # Small threshold to handle numerical precision
    L_true_binary = np.abs(L_true) > threshold
    L_pred_binary = np.abs(L_pred) > threshold

    # True Positives, False Positives, True Negatives, False Negatives
    TP = np.sum((L_true_binary) & (L_pred_binary))
    FP = np.sum((~L_true_binary) & (L_pred_binary))
    TN = np.sum((~L_true_binary) & (~L_pred_binary))
    FN = np.sum((L_true_binary) & (~L_pred_binary))

    # FNR and FPR
    FNR = FN / (TP + FN) if (TP + FN) > 0 else 0
    FPR = FP / (TN + FP) if (TN + FP) > 0 else 0

    return FNR, FPR

In [19]:
gamma = 0.001

# integrator parameters
time = 1000
transient = 14000
# control parameters
mu = 0.001; sigma = 0.001; beta = 4.1;
# system size
n = 200  # system size of connected Kasthuri_net
m = 2  # dimension

# add noise on only x-component
x_noise = np.zeros((n*m))
for i in range(n*m):
    if i%2 == 0:
        x_noise[i] = 1
# coupling
C = 0.1
h = np.eye(m)
h[1,1] = 0;  # x-coupling

G, A, k_in, L, delta = network_generate(n, eta=0.2)
# G, A, L, degrees, delta = undirected_network(n)

x = data_generate(n, m, transient, time, beta, mu, sigma, C, L, delta, h, gamma, x_noise)
# synchronization = sync_error(x)

X, dx = data_split(n, x)
pred_models = predicted_models(n, X, dx)
corr_gt, distance_matrix, s, s_gt, hub_id, ld_id = similarity(n, x, pred_models, k_in)
F_x = local_dynamics_function(n, m, time, x, beta, mu, sigma)
Y_hub, Y = coupling_effect(dx, hub_id, F_x)
L_pred = reconstruction(n, m, time, X, Y)

# Assess reconstruction performance
error = rmse(L, L_pred, C, delta)
FNR, FPR = compute_performance_metrics(L, L_pred)

print(f"Reconstruction Error (RMSE): {error}")
print(f"False Negative Rate (FNR): {FNR}")
print(f"False Positive Rate (FPR): {FPR}")

/Users/artun/Documents/GitHub/network-reconstruction/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.565e-01, tolerance: 2.076e-01
  model = cd_fast.enet_coordinate_descent(
/Users/artun/Documents/GitHub/network-reconstruction/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.344e-01, tolerance: 1.917e-01
  model = cd_fast.enet_coordinate_descent(


one of the low degree nodes: 0
predicted hub: 0
Predicted low degree node 0 has 5.041404946613128 in connection(s).
The real hub is: 18


/Users/artun/Documents/GitHub/network-reconstruction/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/Users/artun/Documents/GitHub/network-reconstruction/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.354e-04, tolerance: 7.398e-05
  model = cd_fast.enet_coordinate_descent(


Reconstruction Error (RMSE): 0.0014611379630447801
False Negative Rate (FNR): 0.6511056511056511
False Positive Rate (FPR): 0.007122471143889072
